In [2]:
from pypdf import PdfReader
import re
import pandas as pd

def remove_after_substr(text, substr):
    index = text.find(substr)
    if index != -1:
        return text[: index + len(substr)]
    return text

def remove_before_substr(text, substr):
    index = text.find(substr)
    if index != -1:
        return text[index :]
    return text

def extract_keys(text):
    root = re.findall(r'^(1\.)\s*(?s:(.*?))(\.\.\. [^\n]*)', text, flags = re.MULTILINE | re.DOTALL)

    if root != []:
        cleaned_root = [(root[0][0].strip(), root[0][1].strip(), root[0][2].strip(), '', '', '')]
        df_root = pd.DataFrame(cleaned_root, columns = ['id', 'left_text', 'left_target', 'sub_id', 'right_text', 'right_target'])
    else:
        df_root = pd.DataFrame()

    pattern = r'^(?:(\d\(\d\)\.)\s*(?s:(.*?))(\.\.\. [^\n]*)|(—\.)\s*(?s:(.*?))(\.\.\. [^\n]*))'
    matches = re.findall(pattern, text, flags = re.MULTILINE | re.DOTALL)
        
    cleaned_matches = [(m[0].strip(), m[1].strip(), m[2].strip(), m[3].strip(), m[4].strip(), m[5].strip()) for m in matches]

    df_couplets = pd.DataFrame(cleaned_matches, columns = ['id', 'left_text', 'left_target', 'sub_id', 'right_text', 'right_target'])

    df = pd.concat([df_root, df_couplets]).reset_index(drop = True)

    return df

def clean_df(df, section_num):

    for i in range(len(df)):
        if df.loc[i, 'sub_id'] != '':
            df.loc[i-1, ['right_text', 'right_target']] = df.loc[i, ['right_text', 'right_target']]

    df = df[df['sub_id'] == ''].drop(columns = 'sub_id').reset_index(drop = True)

    for i in range(len(df)):
        if '(' in df.loc[i, 'id']:
            df.loc[i, 'parent'] = section_num + '-' + re.findall(r'\(([^)]*)\)', df.loc[i, 'id'])[0]

        if df.loc[i, 'id'] != '':
            df.loc[i, 'id'] = section_num + '-' + df.loc[i, 'id'][0]

        df.loc[i, 'left_text'] = re.sub(r'\.{2,}$', '.', df.loc[i, 'left_text']).replace('\n', ' ').replace('- ', '').replace('  ', ' ').replace(' .', '.')
        df.loc[i, 'left_target'] = df.loc[i, 'left_target'].replace('...', '').strip()
        if df.loc[i, 'left_target'].isnumeric():
            df.loc[i, 'left_target'] = section_num + '-' + df.loc[i, 'left_target']
        #else:
        #    df.loc[i, 'left_target'] = df.loc[i, 'left_target'].rsplit(' (')[0]

        df.loc[i, 'right_text'] = re.sub(r'\.{2,}$', '.', df.loc[i, 'right_text']).replace('\n', ' ').replace('- ', '').replace('  ', ' ').replace(' .', '.')
        df.loc[i, 'right_target'] = df.loc[i, 'right_target'].replace('...', '').strip()
        if df.loc[i, 'right_target'].isnumeric():
            df.loc[i, 'right_target'] = section_num + '-' + df.loc[i, 'right_target']
        #else:
        #    df.loc[i, 'right_target'] = df.loc[i, 'right_target'].rsplit(' (')[0]

    return df

def create_terminal_points(df):
    terminal_points = pd.DataFrame(columns = ['id', 'description', 'parent'])
    for i in range(len(df)):
        if '(' in df.loc[i, 'left_target']:
            info = {
                'id' : df.loc[i, 'left_target'], 
                'description': 'For more information see: ' + df.loc[i, 'left_target'],
                'parent' : df.loc[i, 'id']
            }

            terminal_points = pd.concat([terminal_points, pd.DataFrame(info, index = [0])])

        if '(' in df.loc[i, 'right_target']:
            info = {
                'id' : df.loc[i, 'right_target'], 
                'description': 'For more information see: ' + df.loc[i, 'right_target'],
                'parent' : df.loc[i, 'id']
            }

            terminal_points = pd.concat([terminal_points, pd.DataFrame(info, index = [0])]).reset_index(drop = True)

    return terminal_points

In [3]:
class Node:
    def __init__(self, node_id, description=None):
        self.id = node_id
        self.description = description  # Use for terminal nodes
        self.left = None
        self.right = None
        self.left_text = None
        self.right_text = None
        self.parent = None


    def __repr__(self):
        return f"Node({self.id})"
    
    
    def is_terminal(self):
        return self.description is not None
    
class BinaryTree:
    def __init__(self, df_nodes, df_terminal):
        self.nodes = {}
        self.root = self._build_tree(df_nodes, df_terminal)


    def _build_tree(self, df_nodes, df_terminal):
        # Step 1: Create all nodes from df_nodes
        for _, row in df_nodes.iterrows():
            node_id = row['id']
            node = self.nodes.setdefault(node_id, Node(node_id))

            node.left_text = row['left_text']
            node.right_text = row['right_text']

            # Create or get left child
            if pd.notna(row['left_target']):
                left_id = row['left_target']
                left_node = self.nodes.setdefault(left_id, Node(left_id))
                node.left = left_node
                left_node.parent = node

            # Create or get right child
            if pd.notna(row['right_target']):
                right_id = row['right_target']
                right_node = self.nodes.setdefault(right_id, Node(right_id))
                node.right = right_node
                right_node.parent = node

        # Step 2: Add terminal nodes
        for _, row in df_terminal.iterrows():
            term_id = row['id']
            term_node = self.nodes.setdefault(term_id, Node(term_id))
            term_node.description = row['description']

            parent_id = row['parent']
            if parent_id in self.nodes:
                parent_node = self.nodes[parent_id]
                # Attach to parent if not already set
                if parent_node.left is None:
                    parent_node.left = term_node
                elif parent_node.right is None:
                    parent_node.right = term_node
                term_node.parent = parent_node

        # Step 3: Find root (node with no parent)
        for node in self.nodes.values():
            if node.parent is None:
                return node

        raise ValueError("No root node found")
    

    def print_tree(self, node=None, indent=""):
        if node is None:
            node = self.root
        if node:
            print(indent + f"Node ID: {node.id}")
            if node.left_text:
                print(indent + f"  Left condition: {node.left_text}")
            if node.right_text:
                print(indent + f"  Right condition: {node.right_text}")
            if node.description:
                print(indent + f"  Terminal Description: {node.description}")
            if node.left:
                print(indent + "  Left ->")
                self.print_tree(node.left, indent + "    ")
            if node.right:
                print(indent + "  Right ->")
                self.print_tree(node.right, indent + "    ")


    def manual_walk(self):
        current = self.root
        path_history = []

        while current:
            # Check for terminal node FIRST
            if current.is_terminal():
                print(f"\nReached terminal node: {current.description}")
                break

            # Only print choices if it's not a terminal node
            print(f"\nYou are at Node ID: {current.id}")
            if current.left_text:
                print(f"  [L] {current.left_text}")
            if current.right_text:
                print(f"  [R] {current.right_text}")

            choice = input("Choose (L/R): ").strip().lower()
            if choice == 'l':
                if current.left:
                    path_history.append((current.id, 'left', current.left_text))
                    current = current.left
                else:
                    print("No left child! Ending walk.")
                    break
            elif choice == 'r':
                if current.right:
                    path_history.append((current.id, 'right', current.right_text))
                    current = current.right
                else:
                    print("No right child! Ending walk.")
                    break
            else:
                print("Invalid choice. Please enter 'L' or 'R'.")

        # After walk ends, print full history
        print("\n=== Walk History ===")
        for node_id, direction, condition in path_history:
            print(f"At Node {node_id}: went {direction.upper()} because \"{condition}\"")

        if current.is_terminal():
            print(f"Final Node {current.id}: {current.description}")
        else:
            print(f"Ended at Node {current.id if current else 'Unknown'} without reaching a terminal node.")


    def reverse_traversal(self, terminal_node_id):
        RED = "\033[91m"
        RESET = "\033[0m"

        if terminal_node_id not in self.nodes:
            raise ValueError(f"Terminal node {terminal_node_id} not found in tree.")

        path = []
        current = self.nodes[terminal_node_id]

        if not current.is_terminal():
            raise ValueError(f"Node {terminal_node_id} is not a terminal node.")

        # Collect the full path
        while current.parent:
            parent = current.parent

            if parent.left == current:
                direction = 'left'
                selected_text = parent.left_text
                non_selected_text = parent.right_text
                non_selected_direction = 'right'
            elif parent.right == current:
                direction = 'right'
                selected_text = parent.right_text
                non_selected_text = parent.left_text
                non_selected_direction = 'left'
            else:
                raise ValueError(f"Inconsistent tree structure at Node {parent.id}.")

            path.append((parent.id, direction, selected_text, non_selected_direction, non_selected_text))
            current = parent

        path.reverse()

        # Now print the path
        print(f"\n=== Path to terminal node {terminal_node_id} ===")
        for node_id, selected_dir, selected_text, non_dir, non_text in path:
            print(f"At Node {node_id}:")

            if selected_text:
                print(f"  -> {selected_dir.upper()}: {selected_text}")

            if non_text:
                print(f"  {RED}XX {non_dir.upper()}: {non_text}{RESET}")

        print("\n")

        #return path  


    def full_reverse_map(self):
        RED = "\033[91m"
        RESET = "\033[0m"

        # Find all terminal nodes
        terminal_nodes = [node for node in self.nodes.values() if node.is_terminal()]
        if not terminal_nodes:
            print("No terminal nodes found!")
            return

        for terminal in terminal_nodes:
            print(f"\n=== Path to terminal node {terminal.id}: {terminal.description} ===")
            
            path = []
            current = terminal

            while current.parent:
                parent = current.parent

                if parent.left == current:
                    direction = 'left'
                    selected_text = parent.left_text
                    non_selected_text = parent.right_text
                    non_selected_direction = 'right'
                elif parent.right == current:
                    direction = 'right'
                    selected_text = parent.right_text
                    non_selected_text = parent.left_text
                    non_selected_direction = 'left'
                else:
                    raise ValueError(f"Inconsistent tree structure at Node {parent.id}.")

                path.append((parent.id, direction, selected_text, non_selected_direction, non_selected_text))
                current = parent

            path.reverse()

            for node_id, selected_dir, selected_text, non_dir, non_text in path:
                print(f"At Node {node_id}:")

                if selected_text:
                    print(f"  -> {selected_dir.upper()}: {selected_text}")

                if non_text:
                    print(f"  {RED}XX {non_dir.upper()}: {non_text}{RESET}")

            print("\n")

In [4]:
path = "../../data/texts/bees-of-the-world.pdf"

reader = PdfReader(path)

p141 = remove_after_substr(reader.pages[140].extract_text(), 'articu-')
p142 = remove_before_substr(reader.pages[141].extract_text(), 'lation')

txt = p141 + p142
x = extract_keys(txt)
xc = clean_df(x, 'Sec.33')
tn = create_terminal_points(xc)

section33 = BinaryTree(xc, tn)

In [6]:
section33.print_tree()

Node ID: Sec.33-1
  Left condition: Labial palpus with ﬁrst two segments elongate (Fig. 33-1a), ﬂattened, the last two segments small, usually diverging laterally from axis of ﬁrst two, not ﬂattened, rarely absent; galeal comb absent or rarely weakly indicated; stipital comb and concavity commonly present (Figs. 19-1a, 33-1b); galeal blade elongate, commonly as long as or longer than stipes (Fig. 33-1b); volsella frequently absent or difﬁcult to recognize, rarely with distinct digitus and cuspis [L-T (long-tongued) bees].
  Right condition: Labial palpus with the four segments similar to one another (Fig. 33-1c), or ﬁrst or rarely ﬁrst two elongate but not much ﬂattened; galeal comb commonly present; stipital comb and concavity absent (Fig. 33-1d); galeal blade usually shorter than stipes (Fig. 33-1d); volsella commonly well developed, usually with recognizable digitus and cuspis [S-T (short-tongued) bees].
  Left ->
    Node ID: Sec.33-2
      Left condition: Labrum with basolateral a

In [7]:
section33.reverse_traversal('Apidae (Sec. 85)')


=== Path to terminal node Apidae (Sec. 85) ===
At Node Sec.33-1:
  -> LEFT: Labial palpus with ﬁrst two segments elongate (Fig. 33-1a), ﬂattened, the last two segments small, usually diverging laterally from axis of ﬁrst two, not ﬂattened, rarely absent; galeal comb absent or rarely weakly indicated; stipital comb and concavity commonly present (Figs. 19-1a, 33-1b); galeal blade elongate, commonly as long as or longer than stipes (Fig. 33-1b); volsella frequently absent or difﬁcult to recognize, rarely with distinct digitus and cuspis [L-T (long-tongued) bees].
  XX RIGHT: Labial palpus with the four segments similar to one another (Fig. 33-1c), or ﬁrst or rarely ﬁrst two elongate but not much ﬂattened; galeal comb commonly present; stipital comb and concavity absent (Fig. 33-1d); galeal blade usually shorter than stipes (Fig. 33-1d); volsella commonly well developed, usually with recognizable digitus and cuspis [S-T (short-tongued) bees].
At Node Sec.33-2:
  -> RIGHT: Labrum with bas

In [8]:
section33.full_reverse_map()


=== Path to terminal node Megachilidae (Sec. 75): For more information see: Megachilidae (Sec. 75) ===
At Node Sec.33-1:
  -> LEFT: Labial palpus with ﬁrst two segments elongate (Fig. 33-1a), ﬂattened, the last two segments small, usually diverging laterally from axis of ﬁrst two, not ﬂattened, rarely absent; galeal comb absent or rarely weakly indicated; stipital comb and concavity commonly present (Figs. 19-1a, 33-1b); galeal blade elongate, commonly as long as or longer than stipes (Fig. 33-1b); volsella frequently absent or difﬁcult to recognize, rarely with distinct digitus and cuspis [L-T (long-tongued) bees].
  XX RIGHT: Labial palpus with the four segments similar to one another (Fig. 33-1c), or ﬁrst or rarely ﬁrst two elongate but not much ﬂattened; galeal comb commonly present; stipital comb and concavity absent (Fig. 33-1d); galeal blade usually shorter than stipes (Fig. 33-1d); volsella commonly well developed, usually with recognizable digitus and cuspis [S-T (short-tongu

In [9]:
section33.manual_walk()


You are at Node ID: Sec.33-1
  [L] Labial palpus with ﬁrst two segments elongate (Fig. 33-1a), ﬂattened, the last two segments small, usually diverging laterally from axis of ﬁrst two, not ﬂattened, rarely absent; galeal comb absent or rarely weakly indicated; stipital comb and concavity commonly present (Figs. 19-1a, 33-1b); galeal blade elongate, commonly as long as or longer than stipes (Fig. 33-1b); volsella frequently absent or difﬁcult to recognize, rarely with distinct digitus and cuspis [L-T (long-tongued) bees].
  [R] Labial palpus with the four segments similar to one another (Fig. 33-1c), or ﬁrst or rarely ﬁrst two elongate but not much ﬂattened; galeal comb commonly present; stipital comb and concavity absent (Fig. 33-1d); galeal blade usually shorter than stipes (Fig. 33-1d); volsella commonly well developed, usually with recognizable digitus and cuspis [S-T (short-tongued) bees].

You are at Node ID: Sec.33-3
  [L] Glossa pointed at apex, sometimes with ﬂabellum.
  [R] G